## WFP-01-02-03 Data transformation application - Sentinel-1 backscatter Sigma-0 all polarization

This application is part of the usage of Sentinel-1 complex and backscatter data for detection and quantification of food security related (natural) hazards and land cover/ land use change

### <a name="quicklink">Quick link

* [Objective](#Objective)
* [Data](#Data)
* [Service Definition](#Service-Definition)
* [Parameter Definition](#Parameter-Definition)
* [Runtime Parameter Definition](#Runtime-Parameter-Definition)
* [Workflow](#Workflow)
* [License](#License)

### Objective 

Process Sentinel-1 GRD with:

* Application of orbit file (if available, but should not hold the processing of the product)
* Border noise removal (if necessary)
* Calibration
* (Multi-temporal) Speckle filtering 
* Terrain correction
* Conversion to dB

### Data 

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

All data products are distributed in the SENTINEL Standard Archive Format for Europe (SAFE) format.

Sentinel-1 data products are available in single polarisation (VV or HH) for Wave mode and dual polarisation (VV+VH or HH+HV) and single polarisation (HH or VV) for SM, IW and EW modes.

Level-1 Ground Range Detected (GRD) products consist of focused SAR data that has been detected, multi-looked and projected to ground range using an Earth ellipsoid model. Phase information is lost. The resulting product has approximately square resolution pixels and square pixel spacing with reduced speckle at the cost of reduced geometric resolution.

GRD products can be in one of three resolutions:

* Full Resolution (FR)
* High Resolution (HR)
* Medium Resolution (MR).

The resolution is dependent upon the amount of multi-looking performed. Level-1 GRD products are available in MR and HR for IW and EW modes, MR for WV mode and MR, HR and FR for SM mode.

### Service Definition 

In [1]:
service = dict([('title', 'WFP-01-02-03 Sentinel-1 backscatter all polarization'),
                ('abstract', 'WFP-01-02-03 Data transformation application - Sentinel-1 backscatter all polarization'),
                ('id', 'ewf-wfp-01-02-03')])

### Parameter Definition 

**Speckle-Filter filterSizeX**

In [2]:
filterSizeX = dict([('id', 'filterSizeX'),
               ('value', '5'),
               ('title', 'Speckle-Filter filterSizeX'),
               ('abstract', 'Set the Speckle-Filter filterSizeX (defaults to 5)')])

**Speckle-Filter filterSizeY**

In [3]:
filterSizeY = dict([('id', 'filterSizeY'),
               ('value', '5'),
               ('title', 'Speckle-Filter filterSizeY'),
               ('abstract', 'Set the Speckle-Filter filterSizeY (defaults to 5)')])

**Area of interest**

Define the area of interest using a polygon in Well-Known-Text format

In [4]:
wkt = dict([('id', 'wkt'),
            ('value', 'POLYGON ((32.0572 12.4549, 33.9087 12.4549, 33.9087 10.7344, 32.0572 10.7344, 32.0572 12.4549))'),
            ('title', 'Area of interest in WKT'),
            ('abstract', 'Area of interest using a polygon in Well-Known-Text format')])

### Runtime parameter definition

**Input identifier**

This is the Sentinel-1 GRD product identifier, e.g. **S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33**

In [5]:
input_identifier = 'S1A_IW_GRDH_1SDV_20171209T033259_20171209T033328_019620_02154C_8F85'

**Input reference**

This is the Sentinel-1 GRD product catalogue reference

In [6]:
input_reference = 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_GRDH_1SDV_20171209T033259_20171209T033328_019620_02154C_8F85'

**Data path**

This path defines where the data is staged-in. 

In [7]:
data_path = '/workspace/data'

### Workflow

#### Import the packages required for processing the Sentinel-1 backscatter

In [8]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os
import sys
import glob
sys.path.append('/opt/anaconda/bin/')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import gc
import gdal
import osr
from shapely.geometry import box
import cioppy
import lxml.etree as etree
from shapely.wkt import loads

import gzip
import shutil

#### Read the product

In [9]:
s1meta = "manifest.safe"

input_data_path = glob.glob('%s/%s*' %(data_path,input_identifier))

if os.path.isfile(input_data_path[0]) and (input_data_path[0].endswith('.zip')):
    s1prd = input_data_path[0]
elif os.path.isdir(input_data_path[0]):
    s1prd = glob.glob(os.path.join(input_data_path[0],'**/%s' % s1meta))
else:
    raise ValueError('Unrecognized downloaded product format: %s' % input_data_path[0])

reader = ProductIO.getProductReader("SENTINEL-1")
product = reader.readProductNodes(s1prd, None)

#### ThermalNoiseRemoval step

In [10]:
HashMap = jpy.get_type('java.util.HashMap')    
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

parameters = HashMap()

parameters.put('selectedPolarisations', None)
parameters.put('removeThermalNoise', 'true')
parameters.put('reIntroduceThermalNoise', 'false')

thermal_noise_removal = GPF.createProduct('ThermalNoiseRemoval', parameters, product)

#### Apply-Orbit-File

In [11]:
parameters = HashMap()

parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
parameters.put('polyDegree', '3')
parameters.put('continueOnFail', 'false')

apply_orbit_file = GPF.createProduct('Apply-Orbit-File', parameters, thermal_noise_removal)


#### Calibration

In [12]:
parameters = HashMap()

parameters.put('auxFile', 'Product Auxiliary File')
parameters.put('outputImageInComplex', 'false')
parameters.put('outputImageScaleInDb', 'false')
parameters.put('createGammaBand', 'false')
parameters.put('createBetaBand', 'false')
parameters.put('selectedPolarisations', None)
parameters.put('outputSigmaBand', 'true')
parameters.put('outputGammaBand', 'false')
parameters.put('outputBetaBand', 'flase')

calibration = GPF.createProduct('Calibration', parameters, apply_orbit_file)


#### Speckle-Filter

In [13]:
parameters = HashMap()

parameters.put('sourceBands', None)
parameters.put('filter', 'Lee')
parameters.put('filterSizeX', filterSizeX['value'])
parameters.put('filterSizeY', filterSizeY['value'])
parameters.put('dampingFactor', '2')
parameters.put('estimateENL', 'true')
parameters.put('enl', '1.0')
parameters.put('numLooksStr', '1')
parameters.put('targetWindowSizeStr', '3x3')
parameters.put('sigmaStr', '0.9')
parameters.put('anSize', '50')

speckle_filter = GPF.createProduct('Speckle-Filter', parameters, calibration)


#### Terrain-Correction

In [14]:
parameters = HashMap()

parameters.put('sourceBands', None)
parameters.put('demName', 'SRTM 3Sec')
parameters.put('externalDEMFile', '')
parameters.put('externalDEMNoDataValue', '0.0')
parameters.put('externalDEMApplyEGM', 'true')
parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION')
parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
parameters.put('pixelSpacingInMeter', '10.0')
#parameters.put('pixelSpacingInDegree', '8.983152841195215E-5')
parameters.put('mapProjection', 'AUTO:42001')
parameters.put('nodataValueAtSea', 'true')
parameters.put('saveDEM', 'false')
parameters.put('saveLatLon', 'false')
parameters.put('saveIncidenceAngleFromEllipsoid', 'false')
parameters.put('saveProjectedLocalIncidenceAngle', 'false')
parameters.put('saveSelectedSourceBand', 'true')
parameters.put('outputComplex', 'false')
parameters.put('applyRadiometricNormalization', 'false')
parameters.put('saveSigmaNought', 'false')
parameters.put('saveGammaNought', 'false')
parameters.put('saveBetaNought', 'false')
parameters.put('incidenceAngleForSigma0', 'Use projected local incidence angle from DEM')
parameters.put('incidenceAngleForGamma0', 'Use projected local incidence angle from DEM')
parameters.put('auxFile', 'Latest Auxiliary File')

terrain_correction = GPF.createProduct('Terrain-Correction', parameters, speckle_filter)

#### Linear to dB

In [15]:
parameters = HashMap()

lineartodb = GPF.createProduct('linearToFromdB', parameters, terrain_correction)

In [16]:
','.join(list(lineartodb.getBandNames()))

'Sigma0_VH_db,Sigma0_VV_db'

#### Subset

In [17]:
parameters = HashMap()

parameters.put('sourceBands', ','.join(list(lineartodb.getBandNames())))
#parameters.put('region', '')
parameters.put('geoRegion', wkt['value'])
parameters.put('subSamplingX', '1')
parameters.put('subSamplingY', '1')
parameters.put('fullSwath', 'false')
parameters.put('tiePointGridNames', '')
parameters.put('copyMetadata', 'true')

subset = GPF.createProduct('Subset', parameters, lineartodb)

In [18]:
band_names = list(subset.getBandNames())
band_names

['Sigma0_VH_db', 'Sigma0_VV_db']

#### Garbage collector

In [19]:
thermal_noise_removal = None
apply_orbit_file = None
calibration = None
speckle_filter = None
terrain_correction = None

gc.collect()

0

#### Save the result

In [20]:
output_name = '%s_Sigma0_%s' % (input_identifier, 'all_bands')

In [ ]:
ProductIO.writeProduct(subset, output_name + '.tif', 'GeoTIFF-BigTIFF')

#### Results metadata

In [ ]:
def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % sar)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition_parameters = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    acquisition = etree.SubElement(acquisition_parameters, '{%s}Acquisition' % sar)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)

    polarisation_channels = etree.SubElement(acquisition, '{%s}polarisationChannels' % eop)
    
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    orbit_number.text = metadata['orbitNumber']
    wrs_longitude_grid.text = metadata['wrsLongitudeGrid']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)

#### Get the result WKT

In [ ]:
src = gdal.Open(output_name + '.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
        transform.TransformPoint(min_x, min_y)[1],
        transform.TransformPoint(max_x, max_y)[0],
        transform.TransformPoint(max_x, max_y)[1]).wkt

#### Create the EOP XML metadata file

In [ ]:
ciop = cioppy.Cioppy()
search = ciop.search(end_point=input_reference,
                     params=[],
                     output_fields='enclosure,identifier,startdate,enddate,wkt,orbitNumber,orbitDirection,swathIdentifier,wrsLongitudeGrid',
                     model='EOP')      

# update the content of the metadata with the result values
search[0]['identifier'] = output_name
search[0]['wkt'] = result_wkt

eop_xml = output_name + '.xml'
with open(eop_xml, 'wb') as file:
    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    file.write(eop_metadata(search[0]))

#### Create the properties file for the reproducibility notebooks

In [ ]:
for properties_file in ['result', 'stage-in']:

    if properties_file == 'result':
        title = 'Reproducibility notebook used for generating %s' % output_name
    else: 
        title = 'Reproducibility stage-in notebook for Sentinel-1 data for generating %s' % output_name
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (search[0]['startdate'], search[0]['enddate']))
        file.write('geometry=%s' % (search[0]['wkt']))
        

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.